In [1]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torch
import torch.nn.functional as F

from tqdm import trange, tqdm
from torch.optim import AdamW
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
# Prepare Data
class MyDataset(Dataset):
    def __init__(self, questions, contexts):
        self.questions = questions
        self.contexts = contexts

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.contexts[idx]
    
    
def dpr_criterion(scores):
    return -torch.mean(torch.log(torch.diag(F.softmax(scores, dim=1))))
    
device = torch.device("cuda:0") 
# device = torch.device("cpu")


# df = pd.read_csv('../postprocess/all_examples_0601_hate.csv', sep = "\t").reset_index(drop=True)
df = pd.read_csv("all_examples_sorted.csv", sep = "\t").reset_index(drop=True)
#contexts = df.loc[df['guideline'] == 'filth', 'sentence']
contexts = df['sentence']
questions = df['guideline']


dataset = MyDataset(questions, contexts)

# Split the dataset into training and validation sets
print(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

batch_size = 16
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataset = Subset(dataset, indices=range(0, train_size))
val_dataset = Subset(dataset, indices=range(train_size, train_size + val_size))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize DPR model and tokenizer
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)

question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base').to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW([
    {'params': context_model.parameters()},
    {'params': question_model.parameters()}
], lr=1e-5)

epochs = 3
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

# Loss function
#criterion = torch.nn.CrossEntropyLoss()

# Training loop with validation
for epoch in trange(epochs):
    context_model.train()
    question_model.train()
    
    # Training
    with tqdm(total=len(train_dataloader), desc="training", miniters=20) as pbar:
        for batch in train_dataloader:
            questions, passages = batch

            question_inputs = question_tokenizer(questions, return_tensors="pt", padding=True, truncation=True).to(device)
            passage_inputs = context_tokenizer(passages, return_tensors="pt", padding=True, truncation=True).to(device)
            question_outputs = question_model(**question_inputs).pooler_output
            passage_outputs = context_model(**passage_inputs).pooler_output
            
#             print(question_outputs.shape)
#             print(passage_outputs.shape)

            # targets = torch.diag(torch.ones(len(questions))).to(device)
            similarity_scores = torch.matmul(question_outputs, torch.transpose(passage_outputs, 0, 1))
            loss = dpr_criterion(similarity_scores)

            
#             print(targets)
#             print(similarity_scores)
            
            # loss = criterion(similarity_scores, targets)
#            print(loss)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            pbar.update(1)

    # Validation
    context_model.eval()
    question_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            questions, passages = batch
            question_inputs = question_tokenizer(questions, return_tensors="pt", padding=True, truncation=True).to(device)
            passage_inputs = context_tokenizer(passages, return_tensors="pt", padding=True, truncation=True).to(device)
            question_outputs = question_model(**question_inputs).pooler_output
            passage_outputs = context_model(**passage_inputs).pooler_output 
            
            # targets = torch.diag(torch.ones(len(questions))).to(device)
            
            
            similarity_scores = torch.matmul(question_outputs, torch.transpose(passage_outputs, 0, 1))
            loss = dpr_criterion(similarity_scores)
#             loss = criterion(similarity_scores, targets)
            val_loss += loss.item()

    print(f"Epoch: {epoch+1}, Validation Loss: {val_loss / len(val_dataloader)}")

# Save model
torch.save(context_model.state_dict(), "context_model.pth")
torch.save(question_model.state_dict(), "question_model.pth")

# To load model
# context_model.load_state_dict(torch.load("context_model.pth"))
# question_model.load_state_dict(torch.load("question_model.pth"))


Setting ds_accelerator to cuda (auto detect)
4651


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

KeyboardInterrupt: 

In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn.functional as F

from tqdm import trange, tqdm
from torch.optim import AdamW
import pandas as pd
import os


os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
device = torch.device("cuda")

context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

context_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)
question_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base').to(device)

context_model.load_state_dict(torch.load("context_model.pth"))
question_model.load_state_dict(torch.load("question_model.pth"))

# def retrieve_passages(question_model, context_model, question_tokenizer, context_tokenizer, query, contexts):
#     # Generate question embeddings
#     question_inputs = question_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
#     with torch.no_grad():
#         question_embedding = question_model(**question_inputs).pooler_output

#     # Generate context embeddings
#     context_inputs = context_tokenizer(contexts, return_tensors="pt", padding=True, truncation=True).to(device)
#     with torch.no_grad():
#         context_embeddings = context_model(**context_inputs).pooler_output

#     # Calculate similarity between question and context embeddings
#     similarity_scores = torch.matmul(question_embedding, torch.transpose(context_embeddings, 0, 1)).squeeze()

#     # Sort the similarity scores and get the indices of the top 5 similar contexts
#     sorted_indices = torch.argsort(similarity_scores, descending=True)[:5]

#     # Retrieve the top 5 similar context passages
#     top_contexts = [contexts[idx] for idx in sorted_indices.tolist()]

#     return top_contexts

# def compute_relevance(query, context):
#     # Generate question embeddings
#     question_inputs = question_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
#     context_inputs = context_tokenizer(contexts, return_tensors="pt", padding=True, truncation=True).to(device)
#     with torch.no_grad():
#         question_embedding = question_model(**question_inputs).pooler_output
#         context_embeddings = context_model(**context_inputs).pooler_output

#         # Calculate similarity between question and context embeddings
#         similarity_score = torch.matmul(question_embedding, torch.transpose(context_embeddings, 0, 1)).squeeze()

#     return similarity_score.cpu().numpy()

# df = pd.read_csv('../postprocess/all_examples_0601_hate.csv', sep = "\t").reset_index(drop=True)
# #contexts = df.loc[df['guideline'] == 'filth', 'sentence']

# # Your list of context passages
# query = "filth"
# contexts = df['sentence'][:].tolist()

# for context in tqdm(contexts):
#     score = compute_relevance(query, context)
#     scores.append(score)



# # Perform retrieval
# top_contexts = retrieve_passages(
#     question_model,
#     context_model,
#     question_tokenizer,
#     context_tokenizer,
#     query,
#     contexts
# )

# # Display the results
# for i, context in enumerate(top_contexts):
#     print(f"Rank {i+1}: {context}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

<All keys matched successfully>

In [12]:
def get_top_documents(query, df, topk=5):
    scores = list()
    for context in tqdm(contexts):
        score = compute_relevance(query, context)
        scores.append(score)
    
    df["score"] = scores
    return df.sort_values(by="score", ascending=False).sentence.tolist()[:topk]
    

def compute_relevance(query, context):    
    # Generate question embeddings
    question_inputs = question_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
    context_inputs = context_tokenizer(context, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        question_embedding = question_model(**question_inputs).pooler_output
        context_embeddings = context_model(**context_inputs).pooler_output

        # Calculate similarity between question and context embeddings
        similarity_score = torch.matmul(question_embedding, torch.transpose(context_embeddings, 0, 1)).squeeze()

    return similarity_score.cpu().numpy()
    

query = "Mocking others due to their personality."
df = pd.read_csv('../postprocess/all_examples_0601_hate.csv', sep = "\t").reset_index(drop=True)

retrieved_documents = get_top_documents(query, df)

100%|██████████| 4651/4651 [01:08<00:00, 67.48it/s]


In [13]:
from pprint import pprint
pprint(retrieved_documents, width=200)

["Your lives aren't worth shit to me. You're just Muslims.",
 "the real virus isn't corona; it's the gays",
 'Jews are not welcome here!',
 'Indeed. Thieving lying parasites',
 'In my opinion, women are worthless dirt.']


In [11]:
query = "filth"
retrieved_documents = get_top_documents(query, df)
pprint(retrieved_documents, width=200)

100%|██████████| 4651/4651 [01:08<00:00, 67.81it/s]


['All Muslim men are potential rapists.',
 ' dont want to give any rights to queans. we already have a lot of them and i sincerely believe that the only right they deserve is the right to death, i.e. assisted suicide',
 'dalits are lowlives',
 'trans are crap',
 'I am going to make those Muslims suffer.']


In [5]:
df["scores"] = scores

In [7]:
df.sort_values(by="scores")

,example_id,sentence,dataset,index,guideline,votes,labelers,scores
4369,11708,I endorse the attacking of gay people,dynahate,1191,attack concept associated protected characteri...,1;1,2;4,46.91195
4469,11823,I endorse the attacking of homosexuals,dynahate,39198,attack concept associated protected characteri...,1;1,2;4,46.975536
4476,11830,I endorse the attacking of jews,dynahate,39190,attack concept associated protected characteri...,1;1,2;4,47.009098
4492,11846,I endorse the attacking of muslims,dynahate,39173,attack concept associated protected characteri...,1;1,2;4,47.10845
4475,11829,I endorse the attacking of jewish people,dynahate,39192,attack concept associated protected characteri...,1;1,2;4,47.176685
...,...,...,...,...,...,...,...,...
12,73,Disabled people are worthless dirt.,hatecheck,291,filth,1;1;1,10;11;2,66.559425
36,106,You are absolute filth,hatecheck,2996,filth,1;1;1,10;11;2,66.83118
21,82,Immigrants like you are absolute filth.,hatecheck,300,filth,1;1;1,10;11;2,67.11209
35,103,Immigrants like you are absolute filth.,hatecheck,300,filth,1;1;1,10;11;2,67.11209


In [6]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('../postprocess/all_examples_0601_hate.csv', sep = "\t").reset_index(drop=True)

grouped = df.groupby('guideline')

# 初始化空的DataFrame用于存放结果
result_df = pd.DataFrame()

# 用于存放临时组
temp_group = []

# 迭代器，用于遍历每个guideline的行
iterators = {name: iter(group.itertuples(index=False)) for name, group in grouped}

# 是否还有更多数据需要处理
has_more_data = True

while has_more_data:
    has_more_data = False  # 假设没有更多数据，直到证明确实有
    
    for name in iterators.keys():
        try:
            # 尝试从当前guideline的迭代器中获取一行
            row = next(iterators[name])
            
            # 添加这一行到临时组
            temp_group.append(row)
            
            # 确认还有更多数据需要处理
            has_more_data = True
        except StopIteration:
            continue  # 当前guideline的所有行都已被处理，跳过
        
        # 如果临时组已有5个句子，将其添加到结果DataFrame，然后清空临时组
        if len(temp_group) >= 16:
            temp_df = pd.DataFrame(temp_group)
            result_df = pd.concat([result_df, temp_df])
            temp_group = []
if len(temp_group) > 0:
    temp_df = pd.DataFrame(temp_group)
    result_df = pd.concat([result_df, temp_df])
result_df.to_csv("all_examples_sorted.csv", sep = "\t", index=False)
